# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark
findspark.init("/home/rodolfo/spark-3.3.1-bin-hadoop3")
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("log_reg_proj").getOrCreate()

22/12/03 16:03:58 WARN Utils: Your hostname, rodolfo-300E5M-300E5L resolves to a loopback address: 127.0.1.1; using 192.168.15.11 instead (on interface wlp3s0)
22/12/03 16:03:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/03 16:03:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/03 16:04:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
df = spark.read.csv("../../data/customer_churn.csv", inferSchema=True, header=True)

In [6]:
df.show(5)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|  Cynthia Norton|37.0|    

In [36]:
df_select = df.select(["Age", "Total_Purchase", "Account_Manager", "Years", "Num_Sites", "Churn"]).na.drop()

In [37]:
from pyspark.ml.feature import VectorAssembler

In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

In [38]:
assembler = VectorAssembler(
    inputCols=["Age", "Total_Purchase", "Account_Manager", "Years", "Num_Sites"],
    outputCol="features"
)

In [39]:
pipeline = Pipeline(
    stages=[
        assembler,
        LogisticRegression(featuresCol="features", labelCol="Churn")
    ]
)

In [40]:
train, test = df_select.randomSplit([0.7, 0.3])

In [41]:
logistic_regression_model = pipeline.fit(train)

In [42]:
test_pred = logistic_regression_model.transform(test)

In [43]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [46]:
auc = BinaryClassificationEvaluator(labelCol="Churn", rawPredictionCol="probability").evaluate(test_pred)

In [47]:
auc

0.9164535379369142